In [ ]:
!pip install neuralforecast transformers gputil


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.9/257.9 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.8/275.8 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.3/819.3 kB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 MB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 79.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 81.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.9 MB/s eta 0:00:0

In [ ]:
from google.colab import drive

drive.mount('/content/drive')
# Access your dataset via '/content/drive/My Drive/filtered_data.csv'


Mounted at /content/drive


In [ ]:
import os
import torch
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler , StandardScaler , LabelEncoder , OneHotEncoder
from neuralforecast import NeuralForecast
from neuralforecast.models import TimeLLM
#from neuralforecast.utils import AirPassengersPanel, augment_calendar_df


/usr/local/lib/python3.11/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [ ]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"


In [ ]:
# %%
data = pd.read_csv('/content/drive/My Drive/filtered_data.csv')


# %%
data[['LCLid','ToU','holiday']] = data[['LCLid','ToU','holiday']].astype('object')


# %%
df = data[['LCLid', 'day','energy_sum','temperature_avg','dewPoint','EnergyClass', 'Acorn_grouped','day_of_week', 'month_of_year','holiday']]

In [ ]:
# Rename columns to match NeuralForecast requirements
df = df.rename(columns={'LCLid': 'unique_id', 'day': 'ds', 'energy_sum': 'y'})

# Ensure 'ds' is a datetime object
df['ds'] = pd.to_datetime(df['ds'])

# Drop unnecessary columns if needed
df = df[['unique_id', 'ds', 'y','temperature_avg', 'day_of_week', 'month_of_year', 'holiday','EnergyClass', 'Acorn_grouped']]

print(df.head())

  unique_id         ds      y  temperature_avg  day_of_week  month_of_year  \
0       131 2012-07-01  6.281           15.025            6              7   
1       131 2012-07-02  7.441           17.120            0              7   
2       131 2012-07-03  8.279           18.695            1              7   
3       131 2012-07-04  5.938           18.765            2              7   
4       131 2012-07-05  6.082           16.740            3              7   

  holiday EnergyClass Acorn_grouped  
0       0      Medium      Affluent  
1       0      Medium      Affluent  
2       0      Medium      Affluent  
3       0      Medium      Affluent  
4       1      Medium      Affluent  


In [ ]:
df = pd.get_dummies(df, columns=['EnergyClass', 'Acorn_grouped'], drop_first=True)

print(df.head())  # Check the one-hot encoded columns


  unique_id         ds      y  temperature_avg  day_of_week  month_of_year  \
0       131 2012-07-01  6.281           15.025            6              7   
1       131 2012-07-02  7.441           17.120            0              7   
2       131 2012-07-03  8.279           18.695            1              7   
3       131 2012-07-04  5.938           18.765            2              7   
4       131 2012-07-05  6.082           16.740            3              7   

  holiday  EnergyClass_Low  EnergyClass_Medium  Acorn_grouped_Affluent  \
0       0            False                True                    True   
1       0            False                True                    True   
2       0            False                True                    True   
3       0            False                True                    True   
4       1            False                True                    True   

   Acorn_grouped_Comfortable  
0                      False  
1                      F

In [ ]:
# Select 5 unique IDs
sampled_ids = df['unique_id'].unique()[:1000]  # Adjust for more/less
df_small = df[df['unique_id'].isin(sampled_ids)].copy()

#print(f"Selected unique IDs: {sampled_ids}")
print(f"Filtered dataset shape: {df_small.shape}")


Filtered dataset shape: (607000, 11)


In [ ]:
# Define prediction horizon and encoder input size
h = 14  # Forecast horizon (14 days)
input_size = 2 * 14  # 12 weeks = 84 days

# Split the data
Y_train_df = df[df['ds'] < (df['ds'].max() - pd.Timedelta(days=h + 14))]  # Training set
Y_val_df = df[(df['ds'] >= (df['ds'].max() - pd.Timedelta(days=h + 14))) &
              (df['ds'] < (df['ds'].max() - pd.Timedelta(days=h)))]  # Validation set
Y_test_df = df[df['ds'] >= (df['ds'].max() - pd.Timedelta(days=h))]  # Test set

print(f"Training Set: {Y_train_df.shape}")
print(f"Validation Set: {Y_val_df.shape}")
print(f"Test Set: {Y_test_df.shape}")

Training Set: (1487772, 3)
Validation Set: (36036, 3)
Test Set: (38610, 3)


In [ ]:
# Prompt for the LLM
prompt_prefix = (
    "The dataset contains daily energy consumption data. "
    "We aim to forecast the next 14 days using the last 168 days of data."
    "There is seasonal trend as colder month the energy consumption is higher and vice versa."
    "On weekend energy consumption is higher"
)

# Define the TimeLLM model
timellm = TimeLLM(
    h=14,
    input_size=168,  # Encoder input size (84 days)
    llm='openai-community/gpt2',  # LLM backend
    prompt_prefix=prompt_prefix,
    stat_exog_list=[],  # Static variables
    hist_exog_list=[],  # Historic variables
    futr_exog_list=[],
    batch_size=1,
    start_padding_enabled=False,
    windows_batch_size=16,
    d_model = 32,
    d_llm= 768,
    d_ff= 128,
    max_steps=10000,
    val_check_steps=500


)

# Initialize NeuralForecast with daily frequency
nf = NeuralForecast(
    models=[timellm],
    freq='D'  # Daily frequency
)

# Fit the model
nf.fit(df=Y_train_df, val_size=14)  # Use the last 14 days of training data for validation

INFO:lightning_fabric.utilities.seed:Seed set to 1
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=10000` reached.


In [ ]:
forecasts = nf.predict(futr_df=Y_test_df)

/usr/local/lib/python3.11/dist-packages/neuralforecast/core.py:904: UserWarning: Dropped 397,000 unused rows from `futr_df`.
  warnings.warn(f"Dropped {dropped_rows:,} unused rows from `futr_df`.")
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

In [ ]:
# Merge on unique_id and ds to align actuals with predictions
aligned = Y_test_df.merge(forecasts, on=['unique_id', 'ds'], how='inner')

# Extract aligned actual and predicted values
y_true_aligned = aligned['y'].values  # Aligned actual values
y_pred_aligned = aligned['TimeLLM'].values  # Aligned predicted values

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

# MAE
mae = mean_absolute_error(y_true_aligned, y_pred_aligned)

# RMSE
rmse = mean_squared_error(y_true_aligned, y_pred_aligned)

# MAPE
mape = np.mean(np.abs((y_true_aligned - y_pred_aligned) / y_true_aligned)) * 100

# SMAPE
smape = np.mean(2 * np.abs(y_true_aligned - y_pred_aligned) / (np.abs(y_true_aligned) + np.abs(y_pred_aligned))) * 100

# Print results
print(f"MAE: {mae}")
print(f"RMSE: {rmse}")
print(f"MAPE: {mape:.2f}%")
print(f"SMAPE: {smape:.2f}%")

MAE: 2.5463159085134133
RMSE: 21.237390178967782
MAPE: inf%
SMAPE: 24.11%


<ipython-input-13-e498feaa3817>:11: RuntimeWarning: divide by zero encountered in divide
  mape = np.mean(np.abs((y_true_aligned - y_pred_aligned) / y_true_aligned)) * 100


In [ ]:
Y_test_df.head()

,unique_id,ds,y,temperature_avg,day_of_week,month_of_year,holiday,EnergyClass_Low,EnergyClass_Medium,Acorn_grouped_Affluent,Acorn_grouped_Comfortable
0,3,2013-12-01,11.853,5.925,6,12,0,False,False,False,False
1,3,2013-12-02,12.641,8.005,0,12,0,False,False,False,False
2,3,2013-12-03,12.181,5.440,1,12,0,False,False,False,False
3,3,2013-12-04,11.682,5.630,2,12,0,False,False,False,False
4,3,2013-12-05,11.578,5.685,3,12,0,False,False,False,False


In [ ]:
forecasts.shape

(36036, 3)

In [ ]:
# Extract actual and predicted values
y_true = Y_test_df['y'].values  # Actual values
y_pred = forecasts['TimeLLM'].values  # Predicted values


In [ ]:
# Merge on unique_id and ds to align actuals with predictions
aligned = Y_test_df.merge(forecasts, on=['unique_id', 'ds'], how='inner')

# Extract aligned actual and predicted values
y_true_aligned = aligned['y'].values  # Aligned actual values
y_pred_aligned = aligned['TimeLLM'].values  # Aligned predicted values


In [ ]:
y_pred_aligned.size

36036

In [ ]:
y_true_aligned.size

36036

In [ ]:
forecasts.head(20)

,unique_id,ds,TimeLLM
0,3,2013-12-01,17.349463
1,3,2013-12-02,17.331451
2,3,2013-12-03,15.659988
3,3,2013-12-04,15.086173
4,3,2013-12-05,15.373342
5,3,2013-12-06,15.547309
6,3,2013-12-07,14.711849
7,3,2013-12-08,15.968003
8,3,2013-12-09,15.098638
9,3,2013-12-10,14.627434
